In [3]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill
from collections import Counter
import os
from datetime import datetime

class JIRAValidator:
    def __init__(self):
        self.input_df = None
        self.export_df = None
        self.field_mapping = {}
        self.value_mapping = {}
        self.include_fields = []
        self.exclude_fields = []
        
    def load_files(self, input_path, export_path, mapping_path):
        """Load all Excel files"""
        try:
            self.input_df = pd.read_excel(input_path, sheet_name='Main')
            
            # Auto-detect export file format
            if export_path.lower().endswith('.csv'):
                self.export_df = pd.read_csv(export_path)
            else:
                self.export_df = pd.read_excel(export_path)
            
            # Load field mappings
            mapping_wb = pd.ExcelFile(mapping_path)
            if 'Field name mapping' in mapping_wb.sheet_names:
                field_map_df = pd.read_excel(mapping_path, sheet_name='Field name mapping')
                self.field_mapping = dict(zip(field_map_df.iloc[:, 0], field_map_df.iloc[:, 1]))
            
            # Load value mappings
            if 'Field Value mapping' in mapping_wb.sheet_names:
                value_map_df = pd.read_excel(mapping_path, sheet_name='Field Value mapping')
                for _, row in value_map_df.iterrows():
                    field_name = row.iloc[0]
                    source_val = row.iloc[1]
                    target_val = row.iloc[2] if len(row) > 2 else ""
                    
                    if field_name not in self.value_mapping:
                        self.value_mapping[field_name] = {}
                    self.value_mapping[field_name][source_val] = target_val
                    
            print(f"✓ Loaded: {len(self.input_df)} input rows, {len(self.export_df)} export rows")
            print(f"✓ Field mappings: {len(self.field_mapping)}")
            print(f"✓ Value mappings: {len(self.value_mapping)} fields")
            return True
            
        except Exception as e:
            print(f"Error loading files: {e}")
            return False
    
    def set_field_filters(self, include=None, exclude=None):
        """Set include/exclude field filters"""
        self.include_fields = include or []
        self.exclude_fields = exclude or []
        
    def get_target_fields(self):
        """Get list of fields to validate based on filters"""
        # Start with input fields
        available_fields = list(self.input_df.columns)
        
        # Apply include filter
        if self.include_fields and 'All' not in self.include_fields:
            fields = [f for f in self.include_fields if f in available_fields]
        else:
            fields = available_fields
            
        # Apply exclude filter
        if self.exclude_fields and 'All' not in self.exclude_fields:
            fields = [f for f in fields if f not in self.exclude_fields]
        elif 'All' in self.exclude_fields:
            fields = []
            
        return fields
    
    def resolve_field_mapping(self, input_field):
        """Get mapped field name for export comparison"""
        return self.field_mapping.get(input_field, input_field)
    
    def apply_value_mapping(self, field_name, value):
        """Apply value mapping if exists"""
        if field_name in self.value_mapping:
            return self.value_mapping[field_name].get(value, value)
        return value
    
    def get_value_counts(self, df, field_name, apply_mapping=False):
        """Get unique value counts for a field"""
        if field_name not in df.columns:
            return {}
            
        # Handle nulls/blanks
        series = df[field_name].fillna('').astype(str)
        counts = series.value_counts().to_dict()
        
        if apply_mapping:
            mapped_counts = {}
            for value, count in counts.items():
                mapped_value = self.apply_value_mapping(field_name, value)
                mapped_counts[mapped_value] = mapped_counts.get(mapped_value, 0) + count
            return mapped_counts
            
        return counts
    
    def compare_field_data(self, input_field, export_field):
        """Compare data between input and export for a specific field"""
        # Get counts from both sources
        input_counts = self.get_value_counts(self.input_df, input_field, apply_mapping=True)
        export_counts = self.get_value_counts(self.export_df, export_field)
        
        # Compare
        all_values = set(input_counts.keys()) | set(export_counts.keys())
        
        matches = []
        mismatches = []
        new_in_export = []
        missing_in_export = []
        
        for value in all_values:
            input_count = input_counts.get(value, 0)
            export_count = export_counts.get(value, 0)
            
            if input_count == export_count and input_count > 0:
                matches.append(f"{value}: {input_count}")
            elif input_count > 0 and export_count == 0:
                missing_in_export.append(f"{value}: {input_count}")
            elif input_count == 0 and export_count > 0:
                new_in_export.append(f"{value}: {export_count}")
            elif input_count != export_count:
                mismatches.append(f"{value}: {input_count}→{export_count}")
        
        # Determine overall status
        status = "PASS" if not mismatches and not new_in_export and not missing_in_export else "FAIL"
        
        return {
            'status': status,
            'matches': matches,
            'mismatches': mismatches,
            'new_in_export': new_in_export,
            'missing_in_export': missing_in_export,
            'input_total': sum(input_counts.values()),
            'export_total': sum(export_counts.values())
        }
    
    def validate_all_fields(self):
        """Validate all target fields and return results"""
        target_fields = self.get_target_fields()
        results = {}
        
        print(f"Validating {len(target_fields)} fields...")
        
        for input_field in target_fields:
            export_field = self.resolve_field_mapping(input_field)
            
            # Check if mapping exists but field missing
            if export_field not in self.export_df.columns:
                results[input_field] = {
                    'status': 'MISSING_FIELD',
                    'export_field': export_field,
                    'error': f"Field '{export_field}' not found in export"
                }
                continue
            
            # Check if mapping is missing
            if input_field not in self.field_mapping and input_field != export_field:
                results[input_field] = {
                    'status': 'NO_MAPPING',
                    'export_field': export_field,
                    'error': f"No mapping defined for '{input_field}'"
                }
                continue
                
            # Perform comparison
            comparison = self.compare_field_data(input_field, export_field)
            comparison['export_field'] = export_field
            results[input_field] = comparison
            
        return results
    
    def generate_report(self, results, output_path):
        """Generate Excel report with color coding"""
        wb = openpyxl.Workbook()
        
        # Summary sheet
        summary_ws = wb.active
        summary_ws.title = "Summary"
        
        # Color fills
        green_fill = PatternFill(start_color="90EE90", end_color="90EE90", fill_type="solid")
        red_fill = PatternFill(start_color="FFB6C1", end_color="FFB6C1", fill_type="solid")
        yellow_fill = PatternFill(start_color="FFFFE0", end_color="FFFFE0", fill_type="solid")
        
        # Summary headers
        headers = ['Input Field', 'Export Field', 'Status', 'Input Total', 'Export Total', 'Issues']
        for col, header in enumerate(headers, 1):
            summary_ws.cell(row=1, column=col, value=header)
        
        # Summary data
        row = 2
        pass_count = fail_count = missing_count = no_mapping_count = 0
        
        for input_field, result in results.items():
            summary_ws.cell(row=row, column=1, value=input_field)
            summary_ws.cell(row=row, column=2, value=result.get('export_field', ''))
            summary_ws.cell(row=row, column=3, value=result['status'])
            
            # Apply color coding
            status_cell = summary_ws.cell(row=row, column=3)
            if result['status'] == 'PASS':
                status_cell.fill = green_fill
                pass_count += 1
                summary_ws.cell(row=row, column=4, value=result.get('input_total', 0))
                summary_ws.cell(row=row, column=5, value=result.get('export_total', 0))
            elif result['status'] == 'FAIL':
                status_cell.fill = red_fill
                fail_count += 1
                summary_ws.cell(row=row, column=4, value=result.get('input_total', 0))
                summary_ws.cell(row=row, column=5, value=result.get('export_total', 0))
                
                # Compile issues
                issues = []
                if result.get('mismatches'):
                    issues.extend([f"Mismatch: {m}" for m in result['mismatches']])
                if result.get('new_in_export'):
                    issues.extend([f"New: {n}" for n in result['new_in_export']])
                if result.get('missing_in_export'):
                    issues.extend([f"Missing: {m}" for m in result['missing_in_export']])
                summary_ws.cell(row=row, column=6, value="; ".join(issues))
                
            else:  # MISSING_FIELD or NO_MAPPING
                status_cell.fill = yellow_fill
                if result['status'] == 'MISSING_FIELD':
                    missing_count += 1
                else:
                    no_mapping_count += 1
                summary_ws.cell(row=row, column=6, value=result.get('error', ''))
            
            row += 1
        
        # Statistics
        stats_ws = wb.create_sheet("Statistics")
        stats_data = [
            ['Metric', 'Count'],
            ['Total Fields Validated', len(results)],
            ['PASSED', pass_count],
            ['FAILED', fail_count],
            ['Missing Fields', missing_count],
            ['No Mapping', no_mapping_count],
            ['Success Rate %', round((pass_count / len(results)) * 100, 2) if results else 0]
        ]
        
        for row_idx, (metric, count) in enumerate(stats_data, 1):
            stats_ws.cell(row=row_idx, column=1, value=metric)
            stats_ws.cell(row=row_idx, column=2, value=count)
        
        # Detailed mismatches
        detail_ws = wb.create_sheet("Detailed_Mismatches")
        detail_headers = ['Field', 'Status', 'Matches', 'Mismatches', 'New in Export', 'Missing in Export']
        for col, header in enumerate(detail_headers, 1):
            detail_ws.cell(row=1, column=col, value=header)
        
        row = 2
        for input_field, result in results.items():
            if result['status'] == 'FAIL':
                detail_ws.cell(row=row, column=1, value=input_field)
                detail_ws.cell(row=row, column=2, value=result['status'])
                detail_ws.cell(row=row, column=3, value="; ".join(result.get('matches', [])))
                detail_ws.cell(row=row, column=4, value="; ".join(result.get('mismatches', [])))
                detail_ws.cell(row=row, column=5, value="; ".join(result.get('new_in_export', [])))
                detail_ws.cell(row=row, column=6, value="; ".join(result.get('missing_in_export', [])))
                row += 1
        
        wb.save(output_path)
        print(f"✓ Report saved: {output_path}")

def main():
    print("JIRA Data Validation Tool")
    print("=" * 40)
    
    # Get file paths
    input_path = input("Enter Input Excel path: ").strip('"')
    export_path = input("Enter JIRA Export Excel path: ").strip('"')
    mapping_path = input("Enter Mapping Excel path: ").strip('"')
    
    # Initialize validator
    validator = JIRAValidator()
    
    if not validator.load_files(input_path, export_path, mapping_path):
        return
    
    # Get field filters
    print("\nField Selection:")
    print("Available fields:", list(validator.input_df.columns))
    
    include_input = input("Include fields (comma-separated, or 'All'): ").strip()
    exclude_input = input("Exclude fields (comma-separated, or 'All'): ").strip()
    
    include_fields = [f.strip() for f in include_input.split(',')] if include_input else []
    exclude_fields = [f.strip() for f in exclude_input.split(',')] if exclude_input else []
    
    validator.set_field_filters(include_fields, exclude_fields)
    
    # Run validation
    print(f"\nRunning validation...")
    results = validator.validate_all_fields()
    
    # Generate report
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"Validation_Report_{timestamp}.xlsx"
    validator.generate_report(results, output_path)
    
    # Print summary
    pass_count = sum(1 for r in results.values() if r['status'] == 'PASS')
    fail_count = sum(1 for r in results.values() if r['status'] == 'FAIL')
    
    print(f"\nValidation Complete:")
    print(f"✓ PASSED: {pass_count}")
    print(f"✗ FAILED: {fail_count}")
    print(f"Success Rate: {(pass_count/len(results)*100):.1f}%")

if __name__ == "__main__":
    main()

JIRA Data Validation Tool


Enter Input Excel path:  "C:\Users\divya.eesarla\Desktop\Automation Testing\Vaka.xlsx"
Enter JIRA Export Excel path:  "C:\Users\divya.eesarla\Desktop\Automation Testing\JIRA.csv"
Enter Mapping Excel path:  "C:\Users\divya.eesarla\Desktop\Automation Testing\Mapping.xlsx"


✓ Loaded: 106 input rows, 106 export rows
✓ Field mappings: 14
✓ Value mappings: 4 fields

Field Selection:
Available fields: ['ID', 'Item ID', 'Name', 'Description', 'DocumentKey', 'GlobalId', 'Item Type', 'Issue Type', 'Created By', 'Assigned To', 'Project Name', 'Component', 'component_level_1', 'component_level_2', 'Tila', 'Prioriteetti', 'Tyyppi', 'Comments']


Include fields (comma-separated, or 'All'):  Prioriteetti,Tyyppi,Comments,Assigned To,Component,Created By
Exclude fields (comma-separated, or 'All'):  component_level_2



Running validation...
Validating 6 fields...
✓ Report saved: Validation_Report_20250712_025506.xlsx

Validation Complete:
✓ PASSED: 0
✗ FAILED: 4
Success Rate: 0.0%
